In [ ]:
!pip install pyvi
import os
from pyvi import ViTokenizer
from gensim.models import Word2Vec
import numpy as np
import warnings
warnings.filterwarnings('ignore')
#import pyemd
import re
import pandas as pd
#import time
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Flatten
from tensorflow.keras import layers , activations , models , preprocessing , utils
from  sklearn.model_selection import train_test_split

In [ ]:
#from os import walk
#mypath='data'
#f = []
#for (dirpath, dirnames, filenames) in walk(mypath):
#    f.extend(filenames)
#    break

In [ ]:
# Hàm đọc file
def read(listDir, specialFile=None):
  listFile = []
  for f in listDir:
    if f[-4:]=='.txt':
      listFile.append(f)

  lines = []
  for filename in listFile:
    if filename != specialFile:
      with open(filename, 'r', encoding='utf-8') as f:
        for x in f:
          lines.append(x.strip('\n')) #bỏ \n cuối mỗi câu
      f.close()
  return lines

In [ ]:
# Test hàm đọc file
listDir = os.listdir()
specialFile = 'VietNam_Stopwords.txt'
lines = read(listDir, specialFile)
print(len(lines))
print(lines[:5])

stopword = read([specialFile])
print(len(stopword))
print(stopword[:5])

5855
['Thích đánh lộn không? __eou__ Ngon nhào vô __eou__', 'Solo yasua không __eou__ Chấp lun 2 mạng đầu __eou__', 'Mai đi picnic không? __eou__ Mai bận học rồi __eou__', 'Mai học ca mấy vậy? __eou__ Mai học ca 3 __eou__', 'Còn tiền không? __eou__ Còn chết liền __eou__']
1942
['a lô', 'a ha', 'ai', 'ai ai', 'ai nấy']


In [ ]:
question=[]
answer=[]
for line in lines:
  EOU = "__eou__"
  j = line.find(EOU)
  if line[j+7:-8].isspace() == False:
    question.append(line[:j])
    answer.append(line[j+8:-8])
print(len(question))
print(len(answer))

5769
5769


In [ ]:
#question =[]
#answer=[]
#for i in range(0,len(f)):
#    linktxt='./data/'+f[i]
#    with open(linktxt,encoding='UTF-8') as file:
#        lines=file.readlines()
#        for line in lines:
#            tmp = line.split('__eou__')
#            question.append(tmp[0].strip())
#            answer.append(tmp[1].strip())

In [ ]:
def processing(sent):
    sent=sent.lower()
    new_sent=re.findall(r'\b\S+\b',sent)
    new_sent=' '.join(new_sent)
    return new_sent

In [ ]:
for i in range(0,len(question)):
    question[i] = processing(question[i])

In [ ]:
listnull=[]
for i in range(0,len(question)):
    if (question[i]==''):
        listnull.append(i)

In [ ]:
question=list(filter(None,question))
answer.remove(answer[listnull[0]])

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(lower=True,split=' ')
tokenizer.fit_on_texts(question)
print(question[0])
print(question[1])
print(question[2])

X = tokenizer.texts_to_sequences(question)
print(X[0])
print(X[1])
print(X[2])

X = pad_sequences(X)
print(X[0])
print(X[1])
print(X[2])
print(X.shape)

Thích đánh lộn không? 
Solo yasua không 
Mai đi picnic không? 
[15, 479, 1373, 2]
[1056, 1374, 2]
[163, 7, 1375, 2]
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0   15  479 1373    2]
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0 

In [ ]:
print("input shape",X.shape)
vocab_size=len(tokenizer.word_index)+1
print('Vocab count',vocab_size)

input shape (5769, 93)
Vocab count 2143


In [ ]:
#lstm
embed_dim=128
hidden_size=200
batch_size=32
model = models.Sequential()
model.add(layers.Embedding(vocab_size,embed_dim,input_length=X.shape[1]))
model.add(layers.LSTM(hidden_size,return_sequences=True))
model.add(layers.LSTM(hidden_size))
#model.add(layers.Dense(128,activation='sigmoid'))
model.add(layers.Dense(5769,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 93, 128)           274304    
_________________________________________________________________
lstm_6 (LSTM)                (None, 93, 200)           263200    
_________________________________________________________________
lstm_7 (LSTM)                (None, 200)               320800    
_________________________________________________________________
dense_3 (Dense)              (None, 5769)              1159569   
Total params: 2,017,873
Trainable params: 2,017,873
Non-trainable params: 0
_________________________________________________________________


In [ ]:
integer_mapping = {x: i for i,x in enumerate(answer)}

vec = [integer_mapping[word] for word in answer]

In [ ]:
Y = tf.keras.utils.to_categorical(vec)

In [67]:
history = model.fit(X, Y,
                    epochs=100,
                    verbose=1,
                    batch_size=32)

Epoch 1/100
181/181 [==============================] - 99s 521ms/step - loss: 8.6564 - accuracy: 0.0000e+00
Epoch 2/100
181/181 [==============================] - 97s 534ms/step - loss: 8.6563 - accuracy: 0.0024
Epoch 3/100
181/181 [==============================] - 96s 528ms/step - loss: 8.4899 - accuracy: 0.0049
Epoch 4/100
181/181 [==============================] - 96s 530ms/step - loss: 8.0756 - accuracy: 0.0081
Epoch 5/100
181/181 [==============================] - 96s 531ms/step - loss: 7.4356 - accuracy: 0.0105
Epoch 6/100
181/181 [==============================] - 96s 531ms/step - loss: 6.6088 - accuracy: 0.0292
Epoch 7/100
181/181 [==============================] - 98s 540ms/step - loss: 5.7529 - accuracy: 0.0688
Epoch 8/100
181/181 [==============================] - 96s 531ms/step - loss: 4.9059 - accuracy: 0.1440
Epoch 9/100
181/181 [==============================] - 95s 527ms/step - loss: 4.1485 - accuracy: 0.2673
Epoch 10/100
181/181 [==============================] - 100s

In [68]:
loss, accuracy = model.evaluate(X, Y, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))

Training Accuracy: 0.8842


In [69]:
test_sent=['Bạn học trường nào?']
test_sen=[processing(test_sent[0])]

test_seq = tokenizer.texts_to_sequences(test_sen)
padded_test_seq = pad_sequences(test_seq,maxlen=93)
y_pred=model.predict_classes(padded_test_seq)


In [70]:
print('Q:   ',test_sent[0])
print('A:   ',answer[y_pred[0]])

Q:    Bạn học trường nào?
A:    Mình học trường Tôn Đức Thắng.


'Mình mua ở Nguyễn Kim'